## What is Node Splitting in a Decision Tree & Why is it Done?

Node splitting, or simply splitting, is the process of dividing a node into multiple sub-nodes to create relatively pure nodes. There are multiple ways of doing this, which can be broadly divided into two categories based on the type of target variable:

1. Continuous Target Variable
    - Reduction in Variance
2. Categorical Target Variable
    - Gini Impurity
    - Information Gain
    - Chi-Square
    
### But how do these features get selected and how a particular threshold or value gets chosen for a feature?



In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

***

# Decision Tree Splitting Method 1: Reduction in Variance
Reduction in Variance is a method for splitting the node used when the target variable is continuous, i.e., regression problems. It is so-called because it uses variance as a measure for deciding the feature on which node is split into child nodes.

$$ variance = \frac{\sum(X-\mu)^2}{N}  $$

Variance is used for calculating the homogeneity of a node. If a node is entirely homogeneous, then the variance is zero.

Here are the steps to split a decision tree using reduction in variance:
1. For each split, individually calculate the variance of each child node
2. Calculate the variance of each split as the weighted average variance of child nodes
3. Select the split with the lowest variance
4. Perform steps 1-3 until completely homogeneous nodes are achieved

We want a regression task for this. So, we have the data for 50 startups, and we want to predict Profit.

In [2]:
# Importing the dataset
df = pd.read_csv('50_Startups.csv')
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
df['State'].value_counts()

California    17
New York      17
Florida       16
Name: State, dtype: int64

In [4]:
df.shape

(50, 5)

In [5]:
round(df['Profit'].var(),2)

1624588173.41

#### Categorical Variable Split example - Splitting on `State`

Let us try a split by a categorical variable ⇒State=Florida.

If we split by State=FL, our tree will look like below:

In [6]:
print(df.loc[df['State']=='Florida',:].shape)
var_left = round(df.loc[df['State']=='Florida','Profit'].var(),2)
var_left

(16, 5)


1267749524.39

In [7]:
print(df.loc[df['State']!='Florida',:].shape)
var_right = round(df.loc[df['State']!='Florida','Profit'].var(),2)
var_right

(34, 5)


1803421190.62

![](images\b1.PNG)

Overall Variance then is just the weighted sums of individual variances:

In [8]:
overall_variance = (16/(16+34))*var_left + (34/(16+34))*var_right
overall_variance

1632006257.4264

In the above example, the overall variance is not equal to the root node - may be attributed to the class imbalance. Another example of calculations showing the vriance match as seen in [analytics vidhya blog - youtube](https://youtu.be/i5wGWV5H7oM)

![](images\av1.PNG)

Now lets compare the variance of left child node and right child node. And  __lesser the variance better the split__. So left child node will be the better split in this case.

In [9]:
df['R&D Spend'].describe()

count        50.000000
mean      73721.615600
std       45902.256482
min           0.000000
25%       39936.370000
50%       73051.080000
75%      101602.800000
max      165349.200000
Name: R&D Spend, dtype: float64



#### Continuous Variable Splits example - Splitting on `R&D Spend` 
We can choose a threshold of __75000__ and create a tree



In [10]:
round(df['Profit'].var(),2)     # root node profit

1624588173.41

In [11]:
# left_node - R&D Spend >= 75000
var_left = round(df.loc[df['R&D Spend']>=75000,'Profit'].var(),2)
var_left

683974653.15

In [12]:
df.loc[df['R&D Spend']>=75000,:].shape

(24, 5)

In [13]:
# right node - R&D Spend < 75000
var_right = round(df.loc[df['R&D Spend'] < 75000,'Profit'].var(),2)
var_right

609920803.41

In [14]:
df.loc[df['R&D Spend'] < 75000,:].shape

(26, 5)

![](images\av2b.PNG)

In [15]:
overall_variance = (24/50)*var_left + (26/50)*var_right
overall_variance

645466651.2851999

In [16]:
var_left > var_right

True

As right child has lower variance than the left child so the split will be done based on the __R&D Spend < 75000__.

In [17]:
df.loc[df['R&D Spend'] < 75000,:].shape

(26, 5)

> Since the `overall_variance(R&D Spend < 75000)< overall_variance(State==Flourida)`, we prefer a split based on R&D Spend than over State. And as we had established before __lesser the variance better the split__.

So we can go on splitting this table based on the low variance value of profit we can obtain by splitting any of the columns - __R&D Spend, Administration, Marketing Spend and State__.

***
# Decision Tree Splitting Method 2: Information gain

__Information Gain I(G)__ is used for splitting the nodes when the target variable is categorical. It works on the concept of the entropy and is given by:

$$ I(G) = 1 - Entropy $$

__Entropy__ is the measure of impurity in a node. __Higher the value of entropy, lower is the purity of the node.__

The entropy of a homogeneous node is zero. Since we subtract entropy from 1, the Information Gain is higher for the purer nodes with a maximum value of 1. Now, let’s take a look at the formula for calculating the entropy:

$$  Entropy = - \sum_{i=1}^np_i\log_{n}{p_i}  $$

where, n is the number of classes and p is the distribution of class in the node.

$$ 0 <= Entropy <= 1 $$

Steps to split a decision tree using Information Gain:

1. For each split, individually calculate the entropy of each child node
2. Calculate the entropy of each split as the weighted average entropy of child nodes
3. Select the split with the lowest entropy or highest information gain
4. Until you achieve homogeneous nodes, repeat steps 1-3


So again talking about the binary case we talked about before. What is the value of Entropy for all the 5 cases from A-E?

![](images\entropy.PNG)

Entropy values work as expected. Maximum for Node C and the minimum for both A and E. We need to choose the node with Minimum Entropy.

__Case C:__ So if we have 2 class labels the most impure situation we can have is when the labels are evenly split between 2 class labels.

We could also see the plot of Entropy for the binary case to verify the above.

![](images\entropy-G.PNG)

***
# Decision Tree Splitting Method 3: Gini Impurity / Information gain

Gini Impurity I(G) is a method for splitting the nodes when the target variable is categorical. It is the most popular and the easiest way to split a decision tree. 

$$ I(G) = 1 - Gini $$

Gini is the probability of correctly labeling a randomly chosen element if it was randomly labeled according to the distribution of labels in the node. The formula for Gini is:

The Gini Impurity value is: